In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import seaborn as sns
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader.data import DataReader
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import matplotlib.pyplot as plt
from collections import defaultdict
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler  
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout, LSTM
from sklearn.metrics import mean_absolute_error
init_notebook_mode(connected=True)
%matplotlib inline

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning:

Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.

Using TensorFlow backend.


In [2]:
#!pip install quandl
import quandl
quandl.ApiConfig.api_key = "oicN8c5p-84MtZtQbXop"
df = quandl.get("BCHARTS/KRAKENEUR", authtoken="oicN8c5p-84MtZtQbXop")

In [3]:
df = df['20140108':'20200315']

In [4]:
df.head(3)

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-08,624.01000,640.0000,600.0,613.47231,331.481650,204958.394492,618.309926
2014-01-09,613.47231,626.9999,590.0,614.22456,583.334331,355744.845866,609.847265
2014-01-10,615.00000,630.0000,605.0,625.73866,323.047379,199327.057887,617.021127


In [5]:
df.tail(3)

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2020-03-13,4392.7,5431.9,3550.0,5113.4,36178.077361,1.710711e+08,4728.585040
2020-03-14,5106.5,5136.1,4575.1,4683.9,7978.129312,3.884550e+07,4868.998067
2020-03-15,4679.2,5389.7,4617.5,4817.7,9477.520521,4.637637e+07,4893.301889


In [6]:
new_df = df.head(3)
new_df = new_df.append(df.tail(3))
new_df

,Open,High,Low,Close,Volume (BTC),Volume (Currency),Weighted Price
Date,,,,,,,
2014-01-08,624.01000,640.0000,600.0,613.47231,331.481650,2.049584e+05,618.309926
2014-01-09,613.47231,626.9999,590.0,614.22456,583.334331,3.557448e+05,609.847265
2014-01-10,615.00000,630.0000,605.0,625.73866,323.047379,1.993271e+05,617.021127
2020-03-13,4392.70000,5431.9000,3550.0,5113.40000,36178.077361,1.710711e+08,4728.585040
2020-03-14,5106.50000,5136.1000,4575.1,4683.90000,7978.129312,3.884550e+07,4868.998067
2020-03-15,4679.20000,5389.7000,4617.5,4817.70000,9477.520521,4.637637e+07,4893.301889


In [7]:
df_dates = pd.to_datetime(df.index) #df.index.values[0]

In [8]:
df_dates = df_dates.strftime('%Y-%m-%d')

In [9]:
df_dates.values

array(['2014-01-08', '2014-01-09', '2014-01-10', ..., '2020-03-13',
       '2020-03-14', '2020-03-15'], dtype=object)

In [10]:
list(df.columns.values)

['Open',
 'High',
 'Low',
 'Close',
 'Volume (BTC)',
 'Volume (Currency)',
 'Weighted Price']

In [11]:
INCREASING_COLOR = '#39ac73'
DECREASING_COLOR = '#ff3333'
colors = []

for i in range(len(df.Close)):
    if i != 0:
        if df.Close[i] > df.Close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)
        
trace1 = Candlestick(x=df_dates,
                       open=df.Open,
                       high=df.High,
                       low=df.Low,
                       close=df.Close,
                       yaxis='y2',
                       name='OHLC (€)')

trace2 = Bar(x=df_dates,
                y=df['Volume (BTC)'],
                marker=dict(color=colors),
                yaxis='y',
                name='Volume (BTC)')

data = [trace1,trace2]
layout = dict(
    autosize=False,
    width=1000,
    height=600,
    title='OHLC (€) + Volume (BTC)',
    yaxis=dict( title='Volume',domain = [0, 0.2], showticklabels = False, autorange = True, fixedrange = False ),
    yaxis2=dict( title='Price in €',domain = [0.2, 0.8], autorange = True, fixedrange = False  ),
    xaxis=dict(
        title='Dates',
        rangeslider=dict(
            visible = False
        ),
        type='date'
    )

)   
config = {'scrollZoom': True}

fig = Figure(data=data, layout=layout)
iplot(fig, config=config, filename='simple_candlestick')

In [12]:
df = df.replace(0.0, np.nan)

In [13]:
df = df.fillna(method='ffill')

In [14]:
df = df.drop(["Volume (Currency)", "Weighted Price"], axis=1)
df.head(3).append(df.tail(3))

,Open,High,Low,Close,Volume (BTC)
Date,,,,,
2014-01-08,624.01000,640.0000,600.0,613.47231,331.481650
2014-01-09,613.47231,626.9999,590.0,614.22456,583.334331
2014-01-10,615.00000,630.0000,605.0,625.73866,323.047379
2020-03-13,4392.70000,5431.9000,3550.0,5113.40000,36178.077361
2020-03-14,5106.50000,5136.1000,4575.1,4683.90000,7978.129312
2020-03-15,4679.20000,5389.7000,4617.5,4817.70000,9477.520521


In [15]:
len(df)

2259

In [16]:
def train_test_split(df, test_size=0.2):
    split_row = len(df) - int(test_size * len(df))
    train_data = df.iloc[:split_row]
    test_data = df.iloc[split_row:]
    return train_data, test_data

In [17]:
train, test = train_test_split(df, test_size=0.2)

In [18]:
lenoftest = len(test)
print(lenoftest)

451


In [19]:
p1 = Scatter(x=df_dates, 
             y=(train['Close']), 
             mode='lines',
             marker=dict(color = 'rgba(101,101,101 ,0.8)'),
             name='Train'
             )

p2 = Scatter(x=df_dates[-lenoftest:], 
             y=(test['Close']),
             mode='lines',
             line=dict(color = 'rgba(0,191,255 ,0.8)'),
             name='Test'
             )

layout = Layout(title='Train vs. Test',
                xaxis=dict(title='Dates',ticks='', showticklabels=True,
                zeroline=False),
                yaxis=dict(title='Price in €',ticks='', showticklabels=True,
                zeroline=False),
                showlegend=True, hovermode='closest')

config = {'scrollZoom': True}

fig = Figure(data=[p1,p2], layout=layout)

iplot(fig, config=config)

In [20]:
def normalise_zero_base(df):
    return df / df.iloc[0] - 1

def normalise_min_max(df):
    return (df - df.min()) / (data.max() - df.min())

In [21]:
def extract_window_data(df, window_len=5, zero_base=True):
    window_data = []
    for idx in range(len(df) - window_len):
        tmp = df[idx: (idx + window_len)].copy()
        if zero_base:
            tmp = normalise_zero_base(tmp)
        window_data.append(tmp.values)
    return np.array(window_data)

In [22]:
def prepare_data(df, train_col, target_col, window_len=10, zero_base=True, test_size=0.2):
    train_data, test_data = train_test_split(df, test_size=test_size)
    X_train = extract_window_data(train_data[train_col], window_len, zero_base)
    X_test = extract_window_data(test_data[train_col], window_len, zero_base)
    y_train = train_data[target_col][window_len:].values
    y_test = test_data[target_col][window_len:].values
    if zero_base:
        y_train = y_train / train_data[target_col][:-window_len].values - 1
        y_test = y_test / test_data[target_col][:-window_len].values - 1

    return train_data, test_data, X_train, X_test, y_train, y_test

In [23]:
#def build_lstm_model(input_data, output_size, neurons=100,
#                     dropout=0.2, loss='mse', optimizer='adam'):
#    model = Sequential()
#    model.add(LSTM(neurons, activation='relu', input_shape=(input_data.shape[1], input_data.shape[2])))
#    model.add(Dropout(dropout))
#    model.add(Dense(units=output_size))
#    
#    model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])
#    return model

In [24]:
#def build_lstm_model2(input_data, output_size, neurons=100,
#                     dropout=0.2, loss='mse', optimizer='adam'):
#    model = Sequential()
#    model.add(LSTM(neurons,return_sequences=True, activation='relu', input_shape=(input_data.shape[1], input_data.shape[2])))
#    model.add(Dropout(dropout))
#    model.add(LSTM(neurons))
#    model.add(Dropout(dropout))
#    model.add(Dense(units=output_size))
#    
#    model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])
#    return model

In [25]:
#def build_lstm_model3(input_data, output_size, neurons=100,
#                     dropout=0.2, loss='mse', optimizer='adam'):
#    model = Sequential()
#    model.add(LSTM(neurons,return_sequences=True, activation='relu', input_shape=(input_data.shape[1], input_data.shape[2])))
#    model.add(Dropout(dropout))
#    model.add(LSTM(neurons,return_sequences=True))
#    model.add(Dropout(dropout))
#    model.add(LSTM(neurons))
#    model.add(Dropout(dropout))
#    model.add(Dense(units=output_size))
#    
#    model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])
#    return model

In [26]:
def build_lstm_model4(input_data, output_size, neurons=100,
                     dropout=0.2, loss='mse', optimizer='adam'):
    model = Sequential()
    model.add(LSTM(neurons,return_sequences=True, activation='relu', input_shape=(input_data.shape[1], input_data.shape[2])))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons,return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons,return_sequences=True))
    model.add(Dropout(dropout))
    model.add(LSTM(neurons))
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    
    model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])
    return model

In [27]:
#def build_lstm_model5(input_data, output_size, neurons=100,
#                     dropout=0.2, loss='mse', optimizer='adam'):
#    model = Sequential()
#    model.add(LSTM(neurons,return_sequences=True, activation='relu', input_shape=(input_data.shape[1], input_data.shape[2])))
#    model.add(Dropout(dropout))
#    model.add(LSTM(neurons,return_sequences=True))
#    model.add(Dropout(dropout))
#    model.add(LSTM(neurons,return_sequences=True))
#    model.add(Dropout(dropout))
#    model.add(LSTM(neurons,return_sequences=True))
#    model.add(Dropout(dropout))
#    model.add(LSTM(neurons))
#    model.add(Dropout(dropout))
#    model.add(Dense(units=output_size))
#    
#    model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])
#    return model

In [78]:
window_len = 5
test_size = 0.2
zero_base = True
lstm_neurons = 32
epochs = 50
batch_size = 100
loss = 'mse'
dropout = 0.2
optimizer = 'adam'
targetCol = 'Close'
trainCol = ['Open','High','Low','Close']

In [79]:
train, test, X_train, X_test, y_train, y_test = prepare_data(
    df, trainCol, targetCol, window_len=window_len, zero_base=zero_base, test_size=test_size)

In [80]:
#pipeline experiment test
#model1 = build_lstm_model(
#    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
#    optimizer=optimizer)
#model2 = build_lstm_model2(
#    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
#    optimizer=optimizer)
#model3 = build_lstm_model3(
#    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
#    optimizer=optimizer)
#model4 = build_lstm_model4(
#    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
#    optimizer=optimizer)
#model5 = build_lstm_model5(
#    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
#    optimizer=optimizer)
#
#pipelines = [model1, model2, model3, model4, model5]
#
#pipe_dict = {0: 'build_lstm_model', 1: 'build_lstm_model2', 2: 'build_lstm_model3', 3: 'build_lstm_model4', 4: 'build_lstm_model5'}
#
#for pipe in pipelines:
#    pipe.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=1, shuffle=True)
#
#for idx, val in enumerate(pipelines):
#    score = val.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
#    print('%s pipeline test accuracy: %.5f' % (pipe_dict[idx], score[0]))
#    
#best_acc = 1.0
#best_clf = 0
#best_pipe = ''
#for idx, val in enumerate(pipelines):
#    score = val.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
#    if score[0] < best_acc:
#        best_acc = score[0]
#        best_pipe = val
#        best_clf = idx
#print('Best accuracy: %s' % pipe_dict[best_clf])    

In [81]:
#method_name = pipe_dict[best_clf] # set by the command line options
#possibles = globals().copy()
#possibles.update(locals())
#method = possibles.get(method_name)

In [192]:
finalmodel = build_lstm_model4(
    X_train, output_size=1, neurons=lstm_neurons, dropout=dropout, loss=loss,
    optimizer=optimizer)
history = finalmodel.fit(
    X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, verbose=1, shuffle=True)

Train on 1622 samples, validate on 181 samples
Epoch 1/50
1622/1622 [==============================] - 14s 9ms/step - loss: 0.0075 - acc: 0.0271 - val_loss: 0.0067 - val_acc: 0.0000e+00
Epoch 2/50
1622/1622 [==============================] - 1s 771us/step - loss: 0.0070 - acc: 0.0271 - val_loss: 0.0057 - val_acc: 0.0000e+00
Epoch 3/50
1622/1622 [==============================] - 1s 833us/step - loss: 0.0058 - acc: 0.0271 - val_loss: 0.0044 - val_acc: 0.0000e+00
Epoch 4/50
1622/1622 [==============================] - 1s 820us/step - loss: 0.0049 - acc: 0.0271 - val_loss: 0.0036 - val_acc: 0.0000e+00
Epoch 5/50
1622/1622 [==============================] - 1s 841us/step - loss: 0.0047 - acc: 0.0271 - val_loss: 0.0039 - val_acc: 0.0000e+00
Epoch 6/50
1622/1622 [==============================] - 1s 829us/step - loss: 0.0043 - acc: 0.0271 - val_loss: 0.0032 - val_acc: 0.0000e+00
Epoch 7/50
1622/1622 [==============================] - 1s 840us/step - loss: 0.0041 - acc: 0.0271 - val_loss: 0.0

In [193]:
score = finalmodel.evaluate(X_test, y_test, batch_size=batch_size, verbose=1)
print('MSE:', score)

446/446 [==============================] - 0s 534us/step
MSE: [0.0015515046937583273, 0.0]


In [194]:
from math import sqrt 
from sklearn.metrics import mean_squared_error
targets = test[targetCol][window_len:]
preds = finalmodel.predict(X_test).squeeze()
mae = mean_absolute_error(preds, y_test)
print('MAE: %.4f' % mae)
mse = mean_squared_error(preds, y_test)
print('MSE: %.4f' % mse)
rmse = sqrt(mean_squared_error(preds, y_test))
print('RMSE: %.4f' % rmse)

MAE: 0.0252
MSE: 0.0016
RMSE: 0.0394


In [195]:
print(finalmodel.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_57 (LSTM)               (None, 5, 32)             4736      
_________________________________________________________________
dropout_57 (Dropout)         (None, 5, 32)             0         
_________________________________________________________________
lstm_58 (LSTM)               (None, 5, 32)             8320      
_________________________________________________________________
dropout_58 (Dropout)         (None, 5, 32)             0         
_________________________________________________________________
lstm_59 (LSTM)               (None, 5, 32)             8320      
_________________________________________________________________
dropout_59 (Dropout)         (None, 5, 32)             0         
_________________________________________________________________
lstm_60 (LSTM)               (None, 32)                8320      
__________

In [196]:
import pydotplus
import pydot as pyd
from keras.utils.vis_utils import plot_model
plot_model(finalmodel, to_file='finalmodel_plot.png', show_shapes=True, show_layer_names=True)

In [197]:
preds = test[targetCol].values[:-window_len] * (preds + 1)
preds = pd.Series(index=targets.index, data=preds)
lenofytest2 = len(y_test)
print(lenofytest2)
datespreds = df_dates.values[-lenofytest2:]

446


In [198]:
datespreds = datespreds.astype('datetime64',copy=False)
datespreds

array(['2018-12-26', '2018-12-27', '2018-12-28', '2018-12-29',
       '2018-12-30', '2018-12-31', '2019-01-01', '2019-01-02',
       '2019-01-03', '2019-01-04', '2019-01-05', '2019-01-06',
       '2019-01-07', '2019-01-08', '2019-01-09', '2019-01-10',
       '2019-01-11', '2019-01-12', '2019-01-13', '2019-01-14',
       '2019-01-15', '2019-01-16', '2019-01-17', '2019-01-18',
       '2019-01-19', '2019-01-20', '2019-01-21', '2019-01-22',
       '2019-01-23', '2019-01-24', '2019-01-25', '2019-01-26',
       '2019-01-27', '2019-01-28', '2019-01-29', '2019-01-30',
       '2019-01-31', '2019-02-01', '2019-02-02', '2019-02-03',
       '2019-02-04', '2019-02-05', '2019-02-06', '2019-02-07',
       '2019-02-08', '2019-02-09', '2019-02-10', '2019-02-11',
       '2019-02-12', '2019-02-13', '2019-02-14', '2019-02-15',
       '2019-02-16', '2019-02-17', '2019-02-18', '2019-02-19',
       '2019-02-20', '2019-02-21', '2019-02-22', '2019-02-23',
       '2019-02-24', '2019-02-25', '2019-02-26', '2019-

In [199]:
p1 = Scatter(x=datespreds, 
             y=targets, 
             mode='lines',
             marker=dict(color = 'rgba(101,101,101 ,0.8)'),
             name='Actual'
             )

p2 = Scatter(x=datespreds, 
             y=preds,
             mode='lines',
             line=dict(color = 'rgba(0,191,255 ,0.8)'),
             name='Prediction'
             )

layout = Layout(title='Actual Close vs. Prediction Close',
                xaxis=dict(title='Dates',ticks='', showticklabels=True,
                zeroline=False),
                yaxis=dict(title='Price in €',ticks='', showticklabels=True,
                zeroline=False),
                showlegend=True, hovermode='closest')

config = {'scrollZoom': True}

fig = Figure(data=[p1,p2], layout=layout)

iplot(fig, config=config)

In [200]:
INCREASING_COLOR = '#39ac73'
DECREASING_COLOR = '#ff3333'
colors = []

for i in range(len(df.Close)):
    if i != 0:
        if df.Close[i] > df.Close[i-1]:
            colors.append(INCREASING_COLOR)
        else:
            colors.append(DECREASING_COLOR)
    else:
        colors.append(DECREASING_COLOR)

trace1 = Candlestick(x=df_dates,
                       open=df.Open,
                       high=df.High,
                       low=df.Low,
                       close=df.Close,
                       yaxis='y2',
                       name='OHLC')        
        
trace2 = Scatter(x=df_dates,
                 y=df.Close,
                 yaxis='y2',
                 name='Close',
                 marker = dict(
                 size = 2,
                 color = 'rgba(101,101,101 ,0.8)',
                 line = dict(
                     width = 1,)
                 ))

trace3 = Scatter(x=datespreds,
                 y=preds,
                 yaxis='y2',
                 name='Prediction',
                 marker = dict(
                 size = 2,
                 color = 'rgba(0,191,255 ,0.8)',
                 line = dict(
                     width = 1,)
                 ))

data = [trace1,trace2,trace3]
layout = dict(
    autosize=False,
    width=1000,
    height=600,
    title='OHLC with Close and Prediction with all features',
    yaxis=dict( domain = [0, 0.2], showticklabels = False, autorange = True, fixedrange = False ),
    yaxis2=dict( title='Price in €',domain = [0.2, 0.8], autorange = True, fixedrange = False  ),
    xaxis=dict(
        title='Dates',
        rangeslider=dict(
            visible = False
        ),
        type='date'
    )

)   
config = {'scrollZoom': True}

fig = Figure(data=data,layout=layout)
iplot(fig, config=config, filename='simple_candlestick')

In [224]:
epochs = np.arange(1,51)
epochs

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [225]:
trace2 = Scatter(x=epochs,
                 y=history.history['loss'],
                 yaxis='y2',
                 name='MSE (training loss)',
                 marker = dict(
                 size = 2,
                 color = 'rgba(101,101,101 ,0.8)',
                 line = dict(
                     width = 1,)
                 ))

trace3 = Scatter(x=epochs,
                 y=history.history['val_loss'],
                 yaxis='y2',
                 name='MSE (validating loss)',
                 marker = dict(
                 size = 2,
                 color = 'rgba(0,191,255 ,0.8)',
                 line = dict(
                     width = 1,)
                 ))

data = [trace2,trace3]

fig = Figure(data=data)
iplot(fig, config=config, filename='simple_candlestick')